In [ ]:
# Import required libraries
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import pandas as pd
from PIL import Image
import torch
from tqdm import tqdm

# YOLOv8 and custom modules
from ultralytics import YOLO
from src.yolov8_detector import YOLOv8Detector, create_data_yaml
from src.data_preprocessing import YOLODataPreprocessor
from src.evaluation import YOLOEvaluator

# Set style for better plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## 1. Setup and Configuration

Configure the workspace and check available resources.

In [ ]:
# Configuration
CONFIG = {
    'model_size': 'yolov8n',  # n, s, m, l, x
    'device': 'auto',  # auto, cpu, cuda
    'data_dir': 'data',
    'runs_dir': 'runs',
    'batch_size': 16,
    'epochs': 50,
    'img_size': 640,
    'conf_threshold': 0.25,
    'iou_threshold': 0.6
}

# Create directories
for dir_name in [CONFIG['data_dir'], CONFIG['runs_dir']]:
    Path(dir_name).mkdir(exist_ok=True)

print("Configuration:")
for key, value in CONFIG.items():
    print(f"  {key}: {value}")

# Check GPU availability
if torch.cuda.is_available():
    print(f"\nGPU available: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("\nRunning on CPU")

## 2. Data Preparation

Prepare your dataset for YOLOv8 training.

In [ ]:
# Initialize data preprocessor
preprocessor = YOLODataPreprocessor(CONFIG['data_dir'])

# Define your classes
CLASSES = ['person', 'car', 'truck', 'bus', 'motorcycle', 'bicycle']

print(f"Classes: {CLASSES}")
print(f"Number of classes: {len(CLASSES)}")

# Create sample data.yaml (modify paths according to your dataset)
data_yaml_path = create_data_yaml(
    train_path=f"{CONFIG['data_dir']}/train/images",
    val_path=f"{CONFIG['data_dir']}/val/images",
    test_path=f"{CONFIG['data_dir']}/test/images",
    classes=CLASSES,
    save_path=f"{CONFIG['data_dir']}/data.yaml"
)

print(f"Data configuration saved to: {data_yaml_path}")

In [ ]:
# Dataset validation
try:
    stats = preprocessor.validate_dataset(
        images_dir=f"{CONFIG['data_dir']}/train/images",
        labels_dir=f"{CONFIG['data_dir']}/train/labels"
    )
    
    print("Dataset Statistics:")
    print(f"  Total images: {stats['total_images']}")
    print(f"  Total labels: {stats['total_labels']}")
    print(f"  Missing labels: {stats['missing_labels']}")
    print(f"  Missing images: {stats['missing_images']}")
    print(f"  Class distribution: {stats['class_distribution']}")
    
except Exception as e:
    print(f"Dataset validation failed: {e}")
    print("Make sure your dataset is properly organized!")

In [ ]:
# Visualize sample annotations
try:
    preprocessor.visualize_annotations(
        images_dir=f"{CONFIG['data_dir']}/train/images",
        labels_dir=f"{CONFIG['data_dir']}/train/labels",
        class_names=CLASSES,
        num_samples=3
    )
    print("Sample annotations visualized!")
except Exception as e:
    print(f"Visualization failed: {e}")

## 3. Model Training

Train your YOLOv8 model on the prepared dataset.

In [ ]:
# Initialize YOLOv8 detector
detector = YOLOv8Detector(
    model_size=CONFIG['model_size'],
    device=CONFIG['device']
)

print(f"Initialized YOLOv8 detector with {CONFIG['model_size']} on {CONFIG['device']}")

In [ ]:
# Training configuration
training_config = {
    'epochs': CONFIG['epochs'],
    'batch': CONFIG['batch_size'],
    'imgsz': CONFIG['img_size'],
    'workers': 4,
    'patience': 20,
    'save': True,
    'project': CONFIG['runs_dir'],
    'name': f"{CONFIG['model_size']}_training",
    'exist_ok': True,
    'pretrained': True,
    'optimizer': 'AdamW',
    'lr0': 0.001,
    'lrf': 0.01,
    'momentum': 0.937,
    'weight_decay': 0.0005,
    'warmup_epochs': 3.0,
    'warmup_momentum': 0.8,
    'warmup_bias_lr': 0.1,
    'box': 7.5,
    'cls': 0.5,
    'dfl': 1.5,
    'hsv_h': 0.015,
    'hsv_s': 0.7,
    'hsv_v': 0.4,
    'degrees': 0.0,
    'translate': 0.1,
    'scale': 0.5,
    'shear': 0.0,
    'perspective': 0.0,
    'flipud': 0.0,
    'fliplr': 0.5,
    'mosaic': 1.0,
    'mixup': 0.0,
    'copy_paste': 0.0,
}

print("Training configuration:")
for key, value in training_config.items():
    print(f"  {key}: {value}")

In [ ]:
# Start training (this may take a while)
TRAIN_MODEL = False  # Set to True to start training

if TRAIN_MODEL:
    print("Starting model training...")
    results = detector.train(
        data_yaml_path,
        **training_config
    )
    print("Training completed!")
    print(f"Best model saved at: {results.save_dir}/weights/best.pt")
else:
    print("Training skipped. Set TRAIN_MODEL = True to start training.")
    print("Using pretrained model for demonstration.")

## 4. Model Inference

Test your trained model on sample images.

In [ ]:
# Load trained model
model_path = f"{CONFIG['runs_dir']}/train/{CONFIG['model_size']}_training/weights/best.pt"

if os.path.exists(model_path):
    detector.load_model(model_path)
    print(f"Loaded trained model from {model_path}")
else:
    print(f"Trained model not found at {model_path}")
    print("Loading pretrained model for demonstration...")
    detector.load_model()  # Load pretrained model

In [ ]:
# Test inference on sample images
test_images_dir = f"{CONFIG['data_dir']}/test/images"
test_images = list(Path(test_images_dir).glob('*.jpg')) + list(Path(test_images_dir).glob('*.png'))

if test_images:
    # Select a few sample images
    sample_images = test_images[:3]  # First 3 images
    
    fig, axes = plt.subplots(1, len(sample_images), figsize=(15, 5))
    
    for i, img_path in enumerate(sample_images):
        # Run inference
        results = detector.predict(
            str(img_path),
            conf=CONFIG['conf_threshold'],
            iou=CONFIG['iou_threshold'],
            save=False
        )
        
        # Visualize results
        for result in results:
            img = result.plot()
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            if len(sample_images) == 1:
                axes.imshow(img_rgb)
            else:
                axes[i].imshow(img_rgb)
                axes[i].set_title(f"Sample {i+1}")
                axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print(f"Inference completed on {len(sample_images)} sample images")
else:
    print(f"No test images found in {test_images_dir}")

## 5. Model Evaluation

Evaluate your model's performance on the test set.

In [ ]:
# Initialize evaluator
evaluator = YOLOEvaluator(model_path if os.path.exists(model_path) else None)

print("Model evaluator initialized")

In [ ]:
# Run evaluation
RUN_EVALUATION = False  # Set to True to run evaluation

if RUN_EVALUATION:
    print("Running model evaluation...")
    eval_results = evaluator.evaluate_on_dataset(
        data_yaml_path,
        conf=CONFIG['conf_threshold'],
        iou=CONFIG['iou_threshold'],
        save_dir=f"{CONFIG['runs_dir']}/evaluation"
    )
    
    print("\nEvaluation Results:")
    print(f"  mAP50: {eval_results['mAP50']:.4f}")
    print(f"  mAP50-95: {eval_results['mAP50-95']:.4f}")
    print(f"  Precision: {eval_results['precision']:.4f}")
    print(f"  Recall: {eval_results['recall']:.4f}")
    
    print("\nPer-class Metrics:")
    for class_name, metrics in eval_results['class_metrics'].items():
        print(f"  {class_name}:")
        print(f"    Precision: {metrics['precision']:.4f}")
        print(f"    Recall: {metrics['recall']:.4f}")
        print(f"    mAP50: {metrics['mAP50']:.4f}")
else:
    print("Evaluation skipped. Set RUN_EVALUATION = True to run evaluation.")

In [ ]:
# Performance benchmarking
RUN_BENCHMARK = False  # Set to True to run benchmarking

if RUN_BENCHMARK and test_images:
    print("Running performance benchmark...")
    benchmark_results = evaluator.benchmark_inference_speed(
        test_images_dir,
        num_runs=50
    )
    
    print("\nBenchmarking Results:")
    for batch_size, results in benchmark_results.items():
        print(f"  {batch_size}:")
        print(f"    FPS: {results['fps']:.2f}")
        print(f"    Avg time per inference: {results['avg_time_per_inference']*1000:.2f} ms")
else:
    print("Benchmarking skipped. Set RUN_BENCHMARK = True to run benchmarking.")

## 6. Model Export and Deployment

Export your model for deployment.

In [ ]:
# Export model to different formats
EXPORT_MODEL = False  # Set to True to export model

if EXPORT_MODEL:
    print("Exporting model...")
    
    # Export to ONNX
    onnx_path = detector.export_model(
        format='onnx',
        opset=11,
        simplify=True
    )
    print(f"ONNX model exported to: {onnx_path}")
    
    # Export to TensorRT (if on NVIDIA GPU)
    if torch.cuda.is_available():
        trt_path = detector.export_model(
            format='engine',
            device='cuda'
        )
        print(f"TensorRT model exported to: {trt_path}")
    
    print("Model export completed!")
else:
    print("Model export skipped. Set EXPORT_MODEL = True to export model.")

## 7. Interactive Testing

Test your model with custom images or videos.

In [ ]:
# Interactive testing function
def test_on_custom_image(image_path, conf_threshold=0.25, iou_threshold=0.6):
    """
    Test model on a custom image
    
    Args:
        image_path (str): Path to image
        conf_threshold (float): Confidence threshold
        iou_threshold (float): IoU threshold
    """
    if not os.path.exists(image_path):
        print(f"Image not found: {image_path}")
        return
    
    print(f"Testing on: {image_path}")
    
    # Run inference
    results = detector.predict(
        image_path,
        conf=conf_threshold,
        iou=iou_threshold,
        save=False
    )
    
    # Display results
    for result in results:
        img = result.plot()
        plt.figure(figsize=(12, 8))
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.show()
        
        # Print detections
        boxes = result.boxes
        if boxes is not None:
            print(f"Detections: {len(boxes)}")
            for i, box in enumerate(boxes):
                class_id = int(box.cls[0].cpu().numpy())
                confidence = float(box.conf[0].cpu().numpy())
                class_name = CLASSES[class_id] if class_id < len(CLASSES) else f"class_{class_id}"
                print(f"  {i+1}. {class_name}: {confidence:.2f}")
        else:
            print("No detections")

# Example usage
# test_on_custom_image("path/to/your/image.jpg")
print("Interactive testing function ready!")
print("Use: test_on_custom_image('path/to/image.jpg')")

## 8. Summary and Next Steps

This notebook provides a complete workflow for YOLOv8 object detection:

### What we've covered:
1. **Setup and Configuration** - Environment setup and GPU detection
2. **Data Preparation** - Dataset validation and visualization
3. **Model Training** - Training configuration and execution
4. **Model Inference** - Testing on sample images
5. **Model Evaluation** - Performance metrics and benchmarking
6. **Model Export** - Exporting for deployment
7. **Interactive Testing** - Custom image testing

### Next Steps:
- Experiment with different model sizes (yolov8s, yolov8m, etc.)
- Fine-tune hyperparameters for your specific use case
- Implement custom data augmentation strategies
- Deploy the model using the FastAPI service
- Integrate with video streams for real-time detection

### Key Files:
- `src/yolov8_detector.py` - Main detection pipeline
- `src/data_preprocessing.py` - Data preparation utilities
- `src/api.py` - FastAPI service for deployment
- `src/evaluation.py` - Model evaluation and benchmarking

**Happy detecting! 🚀**